In [1]:
import pandas as pd
import regex as re
import os

In [2]:
df = pd.read_pickle('./results_20250722_190837/RESULTS.pkl')

In [3]:
df['GRP'] = df.apply(lambda x: (
    x['PROMPT_ID'], x['RESULT']
),axis=1)

df

,PROMPT_ID,RESULT,RESPONSE,worksheet_name,worksheet_purpose,PROMPT,GRP
0,001,"Here is the complete, user-ready Markdown cont...",response:\nGenerateContentResponse(\n done=...,The Pain Journal,To guide a developer in documenting a recurrin...,You are a Master Curriculum Designer and AI Pe...,"(001, Here is the complete, user-ready Markdow..."
1,002,"Of course. Here is the complete, user-ready Ma...",response:\nGenerateContentResponse(\n done=...,The Evidence Locker,To act as a checklist ensuring the developer g...,You are a Master Curriculum Designer and AI Pe...,"(002, Of course. Here is the complete, user-re..."
2,003,Of course. As a Master Curriculum Designer and...,response:\nGenerateContentResponse(\n done=...,The Rulebook,To help a developer define the absolute constr...,You are a Master Curriculum Designer and AI Pe...,"(003, Of course. As a Master Curriculum Design..."
3,004,"Of course. Here is the complete, user-ready Ma...",response:\nGenerateContentResponse(\n done=...,The Assembly Line,To provide a final checklist and structure for...,You are a Master Curriculum Designer and AI Pe...,"(004, Of course. Here is the complete, user-re..."
4,005,# The Simulation Starter Kit\n\n## Worksheet 1...,response:\nGenerateContentResponse(\n done=...,The Pain Journal,To guide a developer in documenting a recurrin...,You are a Master Curriculum Designer and AI Pe...,"(005, # The Simulation Starter Kit\n\n## Works..."
5,006,"Here is the complete, user-ready Markdown cont...",response:\nGenerateContentResponse(\n done=...,The Evidence Locker,To act as a checklist ensuring the developer g...,You are a Master Curriculum Designer and AI Pe...,"(006, Here is the complete, user-ready Markdow..."
6,007,"Of course. Here is the complete, user-ready Ma...",response:\nGenerateContentResponse(\n done=...,The Rulebook,To help a developer define the absolute constr...,You are a Master Curriculum Designer and AI Pe...,"(007, Of course. Here is the complete, user-re..."
7,008,"Here is the complete, user-ready Markdown cont...",response:\nGenerateContentResponse(\n done=...,The Assembly Line,To provide a final checklist and structure for...,You are a Master Curriculum Designer and AI Pe...,"(008, Here is the complete, user-ready Markdow..."
8,009,"Of course. As a Master Curriculum Designer, I ...",response:\nGenerateContentResponse(\n done=...,The Pain Journal,To guide a developer in documenting a recurrin...,You are a Master Curriculum Designer and AI Pe...,"(009, Of course. As a Master Curriculum Design..."
9,010,"Here is the complete, user-ready Markdown cont...",response:\nGenerateContentResponse(\n done=...,The Evidence Locker,To act as a checklist ensuring the developer g...,You are a Master Curriculum Designer and AI Pe...,"(010, Here is the complete, user-ready Markdow..."


In [4]:
df = df.groupby(['worksheet_name'], as_index=False).agg({
    'worksheet_purpose': 'first',
    'PROMPT_ID': 'min',
    'GRP': lambda x: x.tolist()
})

df


,worksheet_name,worksheet_purpose,PROMPT_ID,GRP
0,The Assembly Line,To provide a final checklist and structure for...,004,"[(004, Of course. Here is the complete, user-r..."
1,The Evidence Locker,To act as a checklist ensuring the developer g...,002,"[(002, Of course. Here is the complete, user-r..."
2,The Pain Journal,To guide a developer in documenting a recurrin...,001,"[(001, Here is the complete, user-ready Markdo..."
3,The Rulebook,To help a developer define the absolute constr...,003,"[(003, Of course. As a Master Curriculum Desig..."


In [5]:
def separate_sections(list_sections, start_sep='START', end_sep='END'):
    content = ''
    for section in list_sections:
        worksheet_id = section[0]
        worksheet = section[1]

        section_content = f'''
=== {start_sep} of worksheet ID: {worksheet_id} ===
{worksheet}
=== {end_sep} of worksheet ID: {worksheet_id} ===
'''
        content += section_content
    return content



df['WORKSHEETS'] = df['GRP'].apply(lambda x: separate_sections(x))
df = df.sort_values(['PROMPT_ID']).reset_index(drop=True)

from pathlib import Path
# create folder
grading_worksheets = './GRADING'
Path(grading_worksheets).mkdir(parents=True, exist_ok=True)

list_worksheets = list(zip(
    df['worksheet_name'], df['WORKSHEETS']

))

for idx, (worksheet_name, worksheet_content) in enumerate(list_worksheets):
    worksheet_path = os.path.join(grading_worksheets, f'{idx+1}_{worksheet_name}.txt')
    with open(worksheet_path, 'w', encoding='utf-8') as file:
        file.write(worksheet_content)
    print(f'Worksheet {idx} saved to {worksheet_path}')



Worksheet 0 saved to ./GRADING\1_The Pain Journal.txt
Worksheet 1 saved to ./GRADING\2_The Evidence Locker.txt
Worksheet 2 saved to ./GRADING\3_The Rulebook.txt
Worksheet 3 saved to ./GRADING\4_The Assembly Line.txt


In [6]:
df

,worksheet_name,worksheet_purpose,PROMPT_ID,GRP,WORKSHEETS
0,The Pain Journal,To guide a developer in documenting a recurrin...,001,"[(001, Here is the complete, user-ready Markdo...",\n=== START of worksheet ID: 001 ===\nHere is ...
1,The Evidence Locker,To act as a checklist ensuring the developer g...,002,"[(002, Of course. Here is the complete, user-r...",\n=== START of worksheet ID: 002 ===\nOf cours...
2,The Rulebook,To help a developer define the absolute constr...,003,"[(003, Of course. As a Master Curriculum Desig...",\n=== START of worksheet ID: 003 ===\nOf cours...
3,The Assembly Line,To provide a final checklist and structure for...,004,"[(004, Of course. Here is the complete, user-r...",\n=== START of worksheet ID: 004 ===\nOf cours...
